1. **Calculate potential evapotranspiration (PET),** or the amount of energy available to vaporize water. The fraction of this energy that is consumed to vaporize water is the actual evapotranspiration (AET). We'll obtain consistent, weekly estimates of both PET and AET from NASA's VIIRS VNP16 product, which uses the same algorithm as MODIS MOD16 [(recall that we used these data in Module 3)](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources).
2. **Define the available soil water.** We can calculate soil water (mm) if we know the **volumetric water content (VWC)** and the **rooting depth.** We'll approximate VWC based on a simplified, annual water balance. We'll use representative rooting depths from the Food and Agriculture Organization (FAO).
3. **Calculate the critical soil water level.** This is the amount of water that is *more than enough* to satisfy the crop. It is also the minimum amount of water required to ensure that all of the available energy to vaporize water (i.e., PET) is used. We'll compute this based on the soil water-holding capacity and other coefficients defined by the FAO.
4. **Calculate the plant available water (PAW);** this is the amount of water the plant can actually use and it is determined by the water balance. We'll have to compute the available soil water at each time step, then add the precipitation (if any), to compute the new PAW at each time step. We'll use precipitation data from NASA.
5. **Compute the Water Requirement Satisfaction Index (WRSI),** an estimate of how much of the plant's water needs have been satisfied. Low values would indicate the plant is water stressed.

**At this point, you're probably very experienced with using the command-line tools on your computer!** You'll see fewer hints in this lesson, but don't forget: to run commands like `cd`, `pixi`, or `snakemake` in this lesson...

#### &#x1FA9F; Windows

**On Windows, you'll want to run these commands in the PowerShell or Command Prompt.**

#### &#x1F34E; &#x1F427; Mac OS/X or GNU/Linux

**On Mac OS/X or GNU/Linux, you'll want to run these commands in the Terminal.**

---

## Getting started with Pixi

To install Pixi and the `pixi` command line tool:

#### &#x1FA9F; Windows

[For Windows, see the instructions on their website.](https://pixi.sh/latest/installation/)

#### &#x1F34E; &#x1F427; Mac OS/X or GNU/Linux

```sh
# If you have curl installed:
curl -fsSL https://pixi.sh/install.sh | sh

# Otherwise:
wget -qO- https://pixi.sh/install.sh | sh
```

Note: In general, you should be suspicious of a command that runs a shell script from the internet! [See the Pixi website for yourself, where you can preview the shell script, if you want to verify its source.](https://pixi.sh/latest/installation/)



### Creating an environment with Pixi

To begin with, we need to create an environment with Pixi. This is similar to [what we did in Module 3](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources/blob/main/notebooks/01_Creating_a_Research_Software_Environment.ipynb), creating a virtual environment with `virtualenv`.

#### &#x1FA9F; &#x1F34E; &#x1F427; All Operating Systems

```sh
# Go to the directory we want to work in
cd demo-M3-project

pixi init
```

When we execute `pixi init` with no further arguments, this creates an environment that has the name of our current directory. If we wanted to create a new directory, with the name of our environment, we could instead write: `pixi init name` where `name` is the name of the environment.

**What happened?** There's now a file called `pixi.toml` in this directory. This file should look something like:

```toml
[workspace]
authors = ["K. Arthur Endsley <arthur.endsley@ntsg.umt.edu>"]
channels = ["conda-forge"]
name = "demo-M5-project"
platforms = ["linux-64"]
version = "0.1.0"

[tasks]

[dependencies]
```

This example is from my computer. The information in `authors` came from my Git installation, because I used `git config` to make sure Git knows my name and the e-mail address I use for my Github account. So, don't be alarmed if you see some personal information in this file!

### Activating our virtual environment

Because we want to do some further work setting up our environment, we need to activate the virtual environment. The command for this is `pixi shell`, because "shell" is another name for the command-line interface to our operating system.

#### &#x1FA9F; &#x1F34E; &#x1F427; All Operating Systems

```sh
pixi shell
```

### Adding software dependencies with Pixi

Let's see how we can use `pixi` to manage software dependencies. Previously, we used `pip` to install Python packages. `pixi` can manage more than just Python packages; it can manage entire software platforms like the Python interpreter itself! 

As an example, let's tell `pixi` that we want to include Python in our project.

#### &#x1FA9F; &#x1F34E; &#x1F427; All Operating Systems

```sh
pixi add python
```

This not only installs the Python interpreter. It tells `pixi` that some of the future things we might `add` will be Python modules and that it should look in certain places (like the Python Package Index or PyPI) to find those things.

How do we install Python packages? It's the same command!

#### &#x1FA9F; &#x1F34E; &#x1F427; All Operating Systems

```sh
# Install the numpy package for Python
pixi add numpy
```

You'll see a few progress bars appear as `pixi` identifies what package we're asking it to install, determines the latest version (since we didn't specify a version), downloads the package, determines the dependencies of this new package and installs those, and ultimately installs the package. This is just as expected, but let's take a look at that `pixi.toml` file again. You should see that the `[dependencies]` section has been updated:

```toml
[dependencies]
numpy = ">=2.3.5,<3"
```

---

## Managing version conflicts

A good package manager needs to be able to handle conflicts between different packages and between different versions of their dependencies. [In Module 3, we discussed how our research project might become dependent on a specific version of a software dependency.](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources/blob/main/notebooks/01_Creating_a_Research_Software_Environment.ipynb) When our project depends on specific versions of one or more packages, that can conflict with the requirements of another one of our software dependencies.

For this project, we need to make sure we're using a version of `numpy` that is *less than* version 2.0. The latest version of `numpy` prior to the version 2.0 release is `1.26.3`. This version of `numpy`, `1.26.3`, has its own software dependencies. Most importantly, `numpy` requires Python, and version `1.26.3` was created to work only with certain versions of Python. **How can we make sure we install both the right version of `numpy` *and* the right version of Python?**

`pixi` and many other package managers handles these situations seamlessly. However, in order to make sure `pixi` has all the required information about potential version conflicts, **we need to tell `pixi` to install all the packages with potential conflicts at the same time.** In this example, that means telling it to install both Python but also a version of `numpy` that is *less than version 2.0.*

#### &#x1FA9F; &#x1F34E; &#x1F427; All Operating Systems

```sh
pixi add python "numpy<2"
```

### Installing the remaining dependencies

**Let's go ahead and install the remaining dependencies we'll need for this lesson.**

#### &#x1FA9F; &#x1F34E; &#x1F427; All Operating Systems

```sh
pixi add earthaccess h5py matplotlib numpy pyproj py4eos rasterio rioxarray xarray
```

**You may have encountered the following error:**

```
Error:   × failed to solve requirements of environment...
  ├─▶   × failed to solve the environment
  │   
  ╰─▶ Cannot solve the request because of: No candidates were found for py4eos *.
```

This happens with some packages that are available in the Python Packaging Index (PyPI) but weren't automatically found. This is often because they aren't indexed by `conda`. To make sure that `pixi` checks PyPI, we need to add the `--pypi` argument:

```sh
pixi add --pypi py4eos
```

---

## Getting started with Snakemake

```sh
pip install snakemake
```

- Making rules
- As in Python `for` loops or function definitions, rules involve a keyword followed by a colon, `:`, and a code block